In [8]:

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

INPUT = "../data/processed/records_2022_clean.csv"
OUT = "../reports/detailed_analysis.md"

Path("../reports").mkdir(parents=True, exist_ok=True)

# Read and prepare data
df = pd.read_csv(INPUT)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')
df['quarter'] = df['date'].dt.to_period('Q')

# Basic metrics
total_rows = len(df)
date_range = f"{df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}"

# Missing data analysis
missing_data = df.isnull().sum()
missing_pct = (missing_data / total_rows * 100).round(2)

# Category analysis
by_category = df['category'].value_counts(dropna=False)
category_status = pd.crosstab(df['category'], df['status'], dropna=False, margins=True)

# Status analysis
by_status = df['status'].value_counts(dropna=False)
status_pct = (by_status / total_rows * 100).round(2)

# Source analysis
by_source = df['source'].value_counts(dropna=False)

# Unit distribution
by_unit = df['unit'].value_counts(dropna=False)

# Temporal analysis
by_month = df.groupby('month').size()
by_quarter = df.groupby('quarter').size()

# Value analysis (numeric only)
df['value_numeric'] = pd.to_numeric(df['value'], errors='coerce')
value_stats = df['value_numeric'].describe()

# Category-specific value analysis
category_value_stats = df.groupby('category')['value_numeric'].agg(['count', 'mean', 'min', 'max'])

# Billing analysis (if billing category exists)
billing_df = df[df['category'] == 'billing'].copy()
if len(billing_df) > 0:
    billing_total = billing_df['value_numeric'].sum()
    billing_by_status = billing_df.groupby('status')['value_numeric'].agg(['count', 'sum', 'mean'])

# Status trends over time
status_by_month = pd.crosstab(df['month'], df['status'])

# Generate report
with open(OUT, "w") as f:
    f.write("# Medical Records Analysis Report\n\n")
    f.write(f"Generated:{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write(f"Dataset:{INPUT}\n\n")
    
    f.write("## Executive Summary\n\n")
    f.write(f"- Total Records: {total_rows:,}\n")
    f.write(f"- Date Range: {date_range}\n")
    f.write(f"- Categories: {df['category'].nunique()}\n")
    f.write(f"- Unique Sources: {df['source'].nunique()}\n\n")
    
    f.write("## Data Quality\n\n")
    f.write("### Missing Values\n\n")
    f.write("| Column | Missing Count | Percentage |\n")
    f.write("|--------|---------------|------------|\n")
    for col in missing_data.index:
        if missing_data[col] > 0:
            f.write(f"| {col} | {missing_data[col]} | {missing_pct[col]}% |\n")
    if missing_data.sum() == 0:
        f.write("| *No missing values* | 0 | 0% |\n")
    f.write("\n")
    
    f.write("## Record Status Distribution\n\n")
    f.write("| Status | Count | Percentage |\n")
    f.write("|--------|-------|------------|\n")
    for status, count in by_status.items():
        status_display = str(status) if pd.notna(status) else "(blank)"
        f.write(f"| {status_display} | {count} | {status_pct[status]:.1f}% |\n")
    f.write("\n")
    
    f.write("## Category Breakdown\n\n")
    f.write("### Records by Category\n\n")
    f.write("| Category | Count | Percentage |\n")
    f.write("|----------|-------|------------|\n")
    for cat, count in by_category.items():
        cat_display = str(cat) if pd.notna(cat) else "(blank)"
        pct = (count / total_rows * 100)
        f.write(f"| {cat_display} | {count} | {pct:.1f}% |\n")
    f.write("\n")
    
    f.write("### Category vs Status Cross-tabulation\n\n")
    f.write(category_status.to_string())
    f.write("\n\n")
    
    f.write("## Data Sources\n\n")
    f.write("| Source | Count | Percentage |\n")
    f.write("|--------|-------|------------|\n")
    for source, count in by_source.items():
        source_display = str(source) if pd.notna(source) else "(blank)"
        pct = (count / total_rows * 100)
        f.write(f"| {source_display} | {count} | {pct:.1f}% |\n")
    f.write("\n")
    
    f.write("## Units of Measurement\n\n")
    f.write("| Unit | Count |\n")
    f.write("|------|-------|\n")
    for unit, count in by_unit.items():
        unit_display = str(unit) if pd.notna(unit) else "(blank)"
        f.write(f"| {unit_display} | {count} |\n")
    f.write("\n")
    
    f.write("## Temporal Analysis\n\n")
    f.write("### Records by Quarter\n\n")
    for quarter, count in by_quarter.items():
        f.write(f"- {quarter}: {count} records\n")
    f.write("\n")
    
    f.write("### Records by Month\n\n")
    for month, count in by_month.items():
        f.write(f"- {month}: {count}\n")
    f.write("\n")
    
    f.write("## Value Analysis\n\n")
    f.write("### Overall Value Statistics\n\n")
    f.write(f"- Count: {value_stats['count']:.0f}\n")
    f.write(f"- Mean: {value_stats['mean']:.2f}\n")
  
    f.write("### Value Statistics by Category\n\n")
    f.write(category_value_stats.to_string())
    f.write("\n\n")
    
    if len(billing_df) > 0:
        f.write("## Billing Analysis\n\n")
        f.write(f"- Total Billing Records: {len(billing_df)}\n")
        f.write(f"- Total Billing Amount: €{billing_total:,.2f}\n")
        f.write(f"- Average Billing: €{billing_df['value_numeric'].mean():,.2f}\n\n")
        f.write("### Billing by Status\n\n")
        f.write(billing_by_status.to_string())
        f.write("\n\n")
    
    f.write("## Status Trends Over Time\n\n")
    f.write(status_by_month.to_string())
    f.write("\n\n")
    
    
    f.write("\n---\n")
    f.write("\n*End of Report*\n")

print(f"Detailed analysis report generated: {OUT}")
print(f"  - Total records analyzed: {total_rows:,}")
print(f"  - Date range: {date_range}")
print(f"  - Categories: {df['category'].nunique()}")
print(f"  - Statuses: {df['status'].nunique()}")

Detailed analysis report generated: ../reports/detailed_analysis.md
  - Total records analyzed: 1,224
  - Date range: 2022-01-01 to 2022-12-31
  - Categories: 10
  - Statuses: 5
